In [5]:
import subprocess
import io
import fetch_data as fd
import numpy as np


categories = ['earn','crude']
numberOfTraining = [10,10]
numberOfTesting = [7,2]


trainData,trainLabels, testData,testLabel = fd.loadData(categories,numberOfTraining,numberOfTesting)
# from subprocess import Popen, PIPE, STDOUT

# Compile the fast_recursive_kernel
test = subprocess.Popen(["g++","fast_recursive_kernel.cpp","-o","fast_recursive_kernel.out"], stdout=subprocess.PIPE)
output_compile = test.communicate()[0]
# print(output_compile.decode('ascii'))


def parseMatrix(matrix):
    num_rows = int(matrix[0])
    num_cols = int(matrix[1])
    K = np.zeros([num_rows,num_cols])
    counter = 2
    for i in range(num_rows):
        for j in range(num_cols):
            K[i,j] = float(matrix[counter])
            counter = counter + 1
    return K

In [6]:
# Test fast_recursive_kernel
n = 2
l = 0.5

def marcus_recursive_kernel(s,t,n,l):
    the_strings = b''
    the_args = bytes(str(len(s)),'ascii')+ b' ' + bytes(str(len(t)),'ascii')+ b' '
    for i in s:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    for i in t:
        the_strings = the_strings + bytes(i,'ascii')
        the_args = the_args + bytes(str(len(i)),'ascii') + b' '
    the_args = the_args + bytes(str(n), 'ascii')+ b' ' + bytes(str(l), 'ascii')
    fast_kernel = subprocess.Popen(["./fast_recursive_kernel.out"], stdin=subprocess.PIPE, stdout=subprocess.PIPE)
    output_test = fast_kernel.communicate(input=b'1 '+the_args+the_strings)[0]
    output_test_list = output_test.decode('utf-8').split()
    K = parseMatrix(output_test_list)
    return K

K = marcus_recursive_kernel(trainData,trainData,n,l)


In [10]:
%timeit marcus_recursive_kernel(trainData,trainData,5,l)

7.6 s ± 115 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
